# Task 1: Introduction

## What is BERT

BERT is a large-scale transformer-based Language Model that can be finetuned for a variety of tasks. 

For more information, the original paper can be found here.
https://arxiv.org/abs/1810.04805

HuggingFace documentation
https://huggingface.co/transformers/model_doc/bert.html

# Task 2: Exploratory Data Analysis and Preprocessing

We will use the SMILE Twitter dataset
https://figshare.com/articles/dataset/smile_annotations_final_csv/3187909/2

In [ ]:
import torch 
import pandas as pd
from tqdm.notebook import tqdm

In [ ]:
df = pd.read_csv('smile-annotaions-final.csv', names = ['id', 'text', 'category'])
df.set_index('id', inplace=True) # 아이디가 데이터가 아닌 인덱스

In [ ]:
df.category.value_counts()

In [ ]:
df = df[~df.category.str.contains('\|')]

In [ ]:
df = df[df.category != 'nocode']

In [ ]:
possible_labels = df.category.unique()

In [ ]:
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
    

In [ ]:
df['label'] = df.category.replace(label_dict)

# Task 3: Training/Validation Split

In [ ]:
#stratified approach 
from skleaern.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    df.index.values, 
    df.label.values, 
    test_size=0.15, 
    random_state = 17, 
    stratify = df.label.values
)

In [ ]:
df['data_type'] = ['not_set']*df.shape[0]

In [ ]:
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [ ]:
df.groupby(['category', 'label', 'data_type']).count()

# Task 4: Loading Tokenizer and Encoding our Data

In [ ]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [ ]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case = True
)

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values, 
    add_special_tokens = True,
    return_attention_mask = True, #언제 문장 끝났는지 알려고
    pad_to_max_length = True,
    max_length = 256,
    return_tensors = 'pt' #pytorch
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text.values, 
    add_special_tokens = True,
    return_attention_mask = True, #언제 문장 끝났는지 알려고
    pad_to_max_length = True,
    max_length = 256,
    return_tensors = 'pt' #pytorch
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_masks']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_masks']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)


In [ ]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_maks_val, labels_val)

# Task 5: Setting up BERT Pretrained Model

In [ ]:
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = len(label_dict),
    output_attentions = False,
    output_hidden_states = False
)

# Task 6: Creating Data Loaders

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentailSampler
batch_size = 4 #32

dataloader_train = DataLoader(
    dataset_train, 
    sampler=RandomSampler(dataset_train),
    batch_size=batch_size
)

dataloader_val = DataLoader(
    dataset_val, 
    sampler=RamdomSampler(dataset_val),
    batch_size = 32
)

# Task 7: Setting Up Optimizer and Scheduler

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [ ]:
optimizer = AdamW(
    model.parameters(),
    lr = 1e-5, #2e-5>5e-5
    eps = 1e-8
)

epochs = 10

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_stemps = 0,
    num_traning_steps = len(dataloader_train)*epochs
)

# Task 8: Defining our Performance Metrics

In [ ]:
import numpy as np
from sklearn.metrics import f1_score

In [ ]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [ ]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v:k for k, v in label_dict.items()}
    preds_flat = np.argmax(preds, axis = 1).flatten()
    labels_flat = labels.flatten()
    
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat == label]
        y_true = labels_flat[labels_flat == label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_reds[y_preds==label])}/{len(y_true)}\n')
        

# Task 9: Creating our Training Loop

Approach adapted from an older version of HuggingFace's run_glue.py script.
Accessible here.
https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

In [ ]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

In [ ]:
def evaluate(dataloader_val):
    
    model.eval()# back progation을 하지 않는 거 빼곤 train과 거의 동일, freeze all weight
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in tqdm(dataloader_val):
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'labels': batch[2]
        }
        
        with torch.no_grad():
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()
        
        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
        
    loss_val_avg = loss_val_total/len(dataloader_val)
    
    predictions = np.concatenate(predictions, axis = 0)
    true_vals = np.concatenate(true_vals, axis=0)
    
    return loss_val_avg, predictions, true_vals


In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    model.train()
    loss_train_total = 0
    progress_bar = tqdm(dataloader_train, desc = 'Epoch {:1d}'.format(epoch), 
                        leave=False, disable=False)
    
    for batch in progress_bar:
        
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {
            'input_ids' : batch[0],
            'attention_mask' : batch[1],
            'labels': batch[2]
        }
        
        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1, 0)
        
        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
    
    torch.save(model.state_dict(), f'Models/BERT_ft_epoch{epoch}')
    
    tqdm.write('\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader)
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (weighted): {val_f1}')
    
    

# Task 10: Loading and Evaluatiog our Model

In [ ]:
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = len(label_dict),
    output_attentions = False,
    output_hidden_states = False
)

In [ ]:
model.to(device)

In [ ]:
model.load_state_dict(
    torch.load('Models/finetuned_bert_epoch_1_gpu_trained.model',
              map_location=torch.device('cpu')))

In [ ]:
_, predictions, true_vals = evaluate(dataloader_val)

In [ ]:
accuracy_per_class(predictions, true_vals)